In [689]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False)
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})

pd.options.display.float_format = '{:20,.2f}'.format

In [690]:
df_train_values_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  uint8   
 2   geo_level_2_id                          260601 non-null  uint16  
 3   geo_level_3_id                          260601 non-null  uint16  
 4   count_floors_pre_eq                     260601 non-null  uint8   
 5   age                                     260601 non-null  uint16  
 6   area_percentage                         260601 non-null  uint16  
 7   height_percentage                       260601 non-null  uint16  
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [691]:
df_train_values_original.count_floors_pre_eq.value_counts()

2    156623
3     55617
1     40441
4      5424
5      2246
6       209
7        39
8         1
9         1
Name: count_floors_pre_eq, dtype: int64

In [692]:
df_train_labels_original.describe()

,building_id,damage_grade
count,"260,601.00","260,601.00"
mean,"525,675.48",2.24
std,"304,545.00",0.61
min,4.00,1.00
25%,"261,190.00",2.00
50%,"525,757.00",2.00
75%,"789,762.00",3.00
max,"1,052,934.00",3.00


In [693]:
#Creo Df para cada tipo de categoria teniendo en cuenta la cantidad de pisos antes del terremoto
#Categorias: 1 piso
#            2 pisos
#            edificios medios = 3, 4 pisos
#            edificios altos = 5, 6, 7, 8, 9 pisos

In [694]:
unPisoDf = df_train_values_original[df_train_values_original["count_floors_pre_eq"] == 1]
unPisoIdDf= pd.DataFrame(unPisoDf.building_id).rename(columns = {0 :"building_id"})

In [695]:
dosPisosDf = df_train_values_original[df_train_values_original["count_floors_pre_eq"] == 2]
dosPisosIdDf= pd.DataFrame(dosPisosDf.building_id).rename(columns = {0 :"building_id"})

In [696]:
edificiosMediosDf = df_train_values_original[df_train_values_original["count_floors_pre_eq"] > 2]
edificiosMediosDf = edificiosMediosDf[edificiosMediosDf["count_floors_pre_eq"] < 5]
edificiosMediosIdDf= pd.DataFrame(edificiosMediosDf.building_id).rename(columns = {0 :"building_id"})

In [697]:
edificiosAltosDf = df_train_values_original[df_train_values_original["count_floors_pre_eq"] > 4]
edificiosAltosIdDf = pd.DataFrame(edificiosAltosDf.building_id).rename(columns = {0 :"building_id"})

In [698]:
# Busco para edificios de 1 piso cual es la superestructura prevalente y veo cual es su danio promedio

In [699]:
# Las superestructuras de piedra y barro(mud_mortar_stone, adobe_mur y stone_flag) comprenden mas de la mitad de los edificios de 1 piso
unPisoBarroPiedraDf = pd.DataFrame(unPisoDf[(unPisoDf.has_superstructure_adobe_mud == True) | (unPisoDf.has_superstructure_mud_mortar_stone == True) | (unPisoDf.has_superstructure_stone_flag == True)].building_id)
unPisoBarroPiedraDf = unPisoBarroPiedraDf.merge(df_train_labels_original, how="inner").drop_duplicates()
unPisoBarroPiedraDf = unPisoBarroPiedraDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_piedra_barro"})

unPisoBarroPiedraPorcentaje = unPisoBarroPiedraDf.div(unPisoBarroPiedraDf.sum(), 1).transpose()

In [700]:
# Los edificios de Madera( Timber y Bamboo) comprenden 1/4 del total
unPisoMaderasDf = pd.DataFrame(unPisoDf[(unPisoDf.has_superstructure_timber == True) | (unPisoDf.has_superstructure_bamboo == True)].building_id)
unPisoMaderasDf = unPisoMaderasDf.merge(df_train_labels_original, how="inner").drop_duplicates()
unPisoMaderasDf = unPisoMaderasDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_madera"})

unPisoMaderasPorcentaje = unPisoMaderasDf.div(unPisoMaderasDf.sum(), 1).transpose()

In [701]:
# Los edificios de Concreto(rc_non_engineered y rc_engineered) comprenden un 10%
unPisoConcretosDf = pd.DataFrame(unPisoDf[(unPisoDf.has_superstructure_rc_non_engineered == True) | (unPisoDf.has_superstructure_rc_engineered == True)].building_id)
unPisoConcretosDf = unPisoConcretosDf.merge(df_train_labels_original, how="inner").drop_duplicates()
unPisoConcretosDf = unPisoConcretosDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_concretos"})

unPisoConcretosPorcentaje = unPisoConcretosDf.div(unPisoConcretosDf.sum(), 1).transpose()

In [702]:
# Los edificios de Cemento( cement_mortar_stone, mud_mortar_brick y cement_moratr_brick) comprenden otro  1/4 del total
unPisoLadrilloDf = pd.DataFrame(unPisoDf[(unPisoDf.has_superstructure_cement_mortar_stone == True) | (unPisoDf.has_superstructure_mud_mortar_brick == True) | (unPisoDf.has_superstructure_cement_mortar_brick == True)].building_id)
unPisoLadrilloDf = unPisoLadrilloDf.merge(df_train_labels_original, how="inner").drop_duplicates()
unPisoLadrilloDf = unPisoLadrilloDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_ladrillo"})

unPisoLadrilloPorcentaje = unPisoLadrilloDf.div(unPisoLadrilloDf.sum(), 1).transpose()

In [703]:
# Los otros edificios son muy pocos, no tiene sentido hacer la comparacion
unPisoOtrosDf = unPisoDf[unPisoDf["has_superstructure_other"] == True].building_id
unPisoOtrosDf = pd.DataFrame(unPisoOtrosDf)
unPisoOtrosDf = unPisoOtrosDf.merge(df_train_labels_original, how="inner").drop_duplicates()

In [704]:
unPisoPorcentajes = pd.concat([unPisoBarroPiedraPorcentaje, unPisoMaderasPorcentaje, unPisoLadrilloPorcentaje, unPisoConcretosPorcentaje], axis=0)
unPisoPorcentajes

damage_grade,1,2,3
superestructura_piedra_barro,0.07,0.53,0.40
superestructura_madera,0.24,0.53,0.23
superestructura_ladrillo,0.36,0.57,0.07
superestructura_concretos,0.52,0.41,0.07


In [705]:
fig = px.bar(unPisoPorcentajes, x=unPisoPorcentajes.columns, y=unPisoPorcentajes.index,
             orientation='h',
             height=400,
             title='Proporcion de daño por uso')
fig.show()

In [706]:
# Las superestructuras de piedra y barro(mud_mortar_stone, adobe_mur y stone_flag) comprenden mas de la mitad de los edificios de 2 piso
dosPisosBarroPiedraDf = pd.DataFrame(dosPisosDf[(dosPisosDf.has_superstructure_adobe_mud == True) | (dosPisosDf.has_superstructure_mud_mortar_stone == True) | (dosPisosDf.has_superstructure_stone_flag == True)].building_id)
dosPisosBarroPiedraDf = dosPisosBarroPiedraDf.merge(df_train_labels_original, how="inner").drop_duplicates()
dosPisosBarroPiedraDf = dosPisosBarroPiedraDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_piedra_barro"})

dosPisosBarroPiedraPorcentaje = dosPisosBarroPiedraDf.div(dosPisosBarroPiedraDf.sum(), 1).transpose()

In [707]:
# Los edificios de Madera( Timber y Bamboo) comprenden 1/4 del total
dosPisosMaderasDf = pd.DataFrame(dosPisosDf[(dosPisosDf.has_superstructure_timber == True) | (dosPisosDf.has_superstructure_bamboo == True)].building_id)
dosPisosMaderasDf = dosPisosMaderasDf.merge(df_train_labels_original, how="inner").drop_duplicates()
dosPisosMaderasDf = dosPisosMaderasDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_madera"})

dosPisosMaderasPorcentaje = dosPisosMaderasDf.div(dosPisosMaderasDf.sum(), 1).transpose()

In [708]:
# Los edificios de Concreto(rc_non_engineered y rc_engineered) comprenden un 10%
dosPisosConcretosDf = pd.DataFrame(dosPisosDf[(dosPisosDf.has_superstructure_rc_non_engineered == True) | (dosPisosDf.has_superstructure_rc_engineered == True)].building_id)
dosPisosConcretosDf = dosPisosConcretosDf.merge(df_train_labels_original, how="inner").drop_duplicates()
dosPisosConcretosDf = dosPisosConcretosDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_concretos"})

dosPisosConcretosPorcentaje = dosPisosConcretosDf.div(dosPisosConcretosDf.sum(), 1).transpose()

In [709]:
# Los edificios de Cemento( cement_mortar_stone, mud_mortar_brick y cement_moratr_brick) comprenden otro  1/4 del total
dosPisosLadrilloDf = pd.DataFrame(dosPisosDf[(dosPisosDf.has_superstructure_cement_mortar_stone == True) | (dosPisosDf.has_superstructure_mud_mortar_brick == True) | (dosPisosDf.has_superstructure_cement_mortar_brick == True)].building_id)
dosPisosLadrilloDf = dosPisosLadrilloDf.merge(df_train_labels_original, how="inner").drop_duplicates()
dosPisosLadrilloDf = dosPisosLadrilloDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_ladrillo"})

dosPisosLadrilloPorcentaje = dosPisosLadrilloDf.div(dosPisosLadrilloDf.sum(), 1).transpose()

In [710]:
dosPisosPorcentajes = pd.concat([dosPisosBarroPiedraPorcentaje, dosPisosMaderasPorcentaje, dosPisosLadrilloPorcentaje, dosPisosConcretosPorcentaje], axis=0)
dosPisosPorcentajes

damage_grade,1,2,3
superestructura_piedra_barro,0.05,0.60,0.35
superestructura_madera,0.12,0.64,0.24
superestructura_ladrillo,0.20,0.63,0.17
superestructura_concretos,0.35,0.53,0.12


In [711]:
fig = px.bar(dosPisosPorcentajes, x=dosPisosPorcentajes.columns, y=dosPisosPorcentajes.index,
             orientation='h',
             height=400,
             title='Proporcion de daño por uso')
fig.show()

In [712]:
# Las superestructuras de piedra y barro(mud_mortar_stone, adobe_mur y stone_flag) comprenden mas de la mitad de los edificios de 2 piso
edificiosMediosBarroPiedraDf = pd.DataFrame(edificiosMediosDf[(edificiosMediosDf.has_superstructure_adobe_mud == True) | (edificiosMediosDf.has_superstructure_mud_mortar_stone == True) | (edificiosMediosDf.has_superstructure_stone_flag == True)].building_id)
edificiosMediosBarroPiedraDf = edificiosMediosBarroPiedraDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosMediosBarroPiedraDf = edificiosMediosBarroPiedraDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_piedra_barro"})

edificiosMediosBarroPiedraPorcentaje = edificiosMediosBarroPiedraDf.div(edificiosMediosBarroPiedraDf.sum(), 1).transpose()

In [713]:
# Los edificios de Madera( Timber y Bamboo) comprenden 1/4 del total
edificiosMediosMaderasDf = pd.DataFrame(edificiosMediosDf[(edificiosMediosDf.has_superstructure_timber == True) | (edificiosMediosDf.has_superstructure_bamboo == True)].building_id)
edificiosMediosMaderasDf = edificiosMediosMaderasDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosMediosMaderasDf = edificiosMediosMaderasDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_madera"})

edificiosMediosMaderasPorcentaje = edificiosMediosMaderasDf.div(edificiosMediosMaderasDf.sum(), 1).transpose()

In [714]:
# Los edificios de Concreto(rc_non_engineered y rc_engineered) comprenden un 10%
edificiosMediosConcretosDf = pd.DataFrame(edificiosMediosDf[(edificiosMediosDf.has_superstructure_rc_non_engineered == True) | (edificiosMediosDf.has_superstructure_rc_engineered == True)].building_id)
edificiosMediosConcretosDf = edificiosMediosConcretosDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosMediosConcretosDf = edificiosMediosConcretosDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_concretos"})

edificiosMediosConcretosPorcentaje = edificiosMediosConcretosDf.div(edificiosMediosConcretosDf.sum(), 1).transpose()

In [715]:
# Los edificios de Cemento( cement_mortar_stone, mud_mortar_brick y cement_moratr_brick) comprenden otro  1/4 del total
edificiosMediosLadrilloDf = pd.DataFrame(edificiosMediosDf[(edificiosMediosDf.has_superstructure_cement_mortar_stone == True) | (edificiosMediosDf.has_superstructure_mud_mortar_brick == True) | (edificiosMediosDf.has_superstructure_cement_mortar_brick == True)].building_id)
edificiosMediosLadrilloDf = edificiosMediosLadrilloDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosMediosLadrilloDf = edificiosMediosLadrilloDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_ladrillo"})

edificiosMediosLadrilloPorcentaje = edificiosMediosLadrilloDf.div(edificiosMediosLadrilloDf.sum(), 1).transpose()

In [716]:
edificiosMediosPorcentajes = pd.concat([edificiosMediosBarroPiedraPorcentaje, edificiosMediosMaderasPorcentaje, edificiosMediosLadrilloPorcentaje, edificiosMediosConcretosPorcentaje], axis=0)
edificiosMediosPorcentajes

damage_grade,1,2,3
superestructura_piedra_barro,0.02,0.52,0.47
superestructura_madera,0.02,0.54,0.44
superestructura_ladrillo,0.10,0.63,0.27
superestructura_concretos,0.44,0.46,0.10


In [717]:
fig = px.bar(edificiosMediosPorcentajes, x=edificiosMediosPorcentajes.columns, y=edificiosMediosPorcentajes.index,
             orientation='h',
             height=400,
             title='Proporcion de daño por uso')
fig.show()

In [718]:
# Las superestructuras de piedra y barro(mud_mortar_stone, adobe_mur y stone_flag) comprenden mas de la mitad de los edificios de 2 piso
edificiosAltosBarroPiedraDf = pd.DataFrame(edificiosAltosDf[(edificiosAltosDf.has_superstructure_adobe_mud == True) | (edificiosAltosDf.has_superstructure_mud_mortar_stone == True) | (edificiosAltosDf.has_superstructure_stone_flag == True)].building_id)
edificiosAltosBarroPiedraDf = edificiosAltosBarroPiedraDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosAltosBarroPiedraDf = edificiosAltosBarroPiedraDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_piedra_barro"})

edificiosAltosBarroPiedraPorcentaje = edificiosAltosBarroPiedraDf.div(edificiosAltosBarroPiedraDf.sum(), 1).transpose()

In [719]:
# Los edificios de Madera( Timber y Bamboo) comprenden 1/4 del total
edificiosAltosMaderasDf = pd.DataFrame(edificiosAltosDf[(edificiosAltosDf.has_superstructure_timber == True) | (edificiosAltosDf.has_superstructure_bamboo == True)].building_id)
edificiosAltosMaderasDf = edificiosAltosMaderasDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosAltosMaderasDf = edificiosAltosMaderasDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_madera"})

edificiosAltosMaderasPorcentaje = edificiosAltosMaderasDf.div(edificiosAltosMaderasDf.sum(), 1).transpose()

In [720]:
# Los edificios de Concreto(rc_non_engineered y rc_engineered) comprenden un 10%
edificiosAltosConcretosDf = pd.DataFrame(edificiosAltosDf[(edificiosAltosDf.has_superstructure_rc_non_engineered == True) | (edificiosAltosDf.has_superstructure_rc_engineered == True)].building_id)
edificiosAltosConcretosDf = edificiosAltosConcretosDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosAltosConcretosDf = edificiosAltosConcretosDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_concretos"})

edificiosAltosConcretosPorcentaje = edificiosAltosConcretosDf.div(edificiosAltosConcretosDf.sum(), 1).transpose()

In [721]:
# Los edificios de Cemento( cement_mortar_stone, mud_mortar_brick y cement_moratr_brick) comprenden otro  1/4 del total
edificiosAltosLadrilloDf = pd.DataFrame(edificiosAltosDf[(edificiosAltosDf.has_superstructure_cement_mortar_stone == True) | (edificiosAltosDf.has_superstructure_mud_mortar_brick == True) | (edificiosAltosDf.has_superstructure_cement_mortar_brick == True)].building_id)
edificiosAltosLadrilloDf = edificiosAltosLadrilloDf.merge(df_train_labels_original, how="inner").drop_duplicates()
edificiosAltosLadrilloDf = edificiosAltosLadrilloDf.groupby(by="damage_grade").agg("count").rename(columns = {"building_id" : "superestructura_ladrillo"})

edificiosAltosLadrilloPorcentaje = edificiosAltosLadrilloDf.div(edificiosAltosLadrilloDf.sum(), 1).transpose()

In [722]:
edificiosAltosPorcentajes = pd.concat([edificiosAltosBarroPiedraPorcentaje, edificiosAltosMaderasPorcentaje, edificiosAltosLadrilloPorcentaje, edificiosAltosConcretosPorcentaje], axis=0)
edificiosAltosPorcentajes

damage_grade,1,2,3
superestructura_piedra_barro,0.01,0.69,0.30
superestructura_madera,0.01,0.72,0.27
superestructura_ladrillo,0.07,0.71,0.23
superestructura_concretos,0.49,0.43,0.08


In [723]:
fig = px.bar(edificiosAltosPorcentajes, x=edificiosAltosPorcentajes.columns, y=edificiosAltosPorcentajes.index,
             orientation='h',
             height=400,
             title='Proporcion de daño por uso')
fig.show()